# 선형회귀 개요

선형 회귀(線型回歸, Linear regression)는 종속 변수 y와 한 개 이상의 독립 변수X와의 선형 상관 관계를 모델링하는 회귀분석 기법. [위키백과](https://ko.wikipedia.org/wiki/%EC%84%A0%ED%98%95_%ED%9A%8C%EA%B7%80)

 - 선형 회귀의 hyper_parameter를 가지고 있는 object
   - Ridge
   - Lasso
   - Elasticnet

## 선형회귀 모델
- 입력 Feature에 가중치(Weight)를 곱하고 편향(bias)를 더해 예측 결과를 출력한다.
- Weight와 bias가 학습대상 Parameter가 된다.

$$
\hat{y_i} = w_1 x_{i1} + w_2 x_{i2}... + w_{p} x_{ip} + b
\\
\hat{y_i} = \mathbf{w}^{T} \cdot \mathbf{X} 
$$

- $\hat{y_i}$: 예측값
- $x$: 특성(feature-컬럼)
- $w$: 가중치(weight), 회귀계수(regression coefficient). 특성이 $\hat{y_i}$ 에 얼마나 영향을 주는지 정도
- $b$: 절편
- $p$: p 번째 특성(feature)/p번째 가중치
- $i$: i번째 관측치(sample)

### Boston DataSet
보스톤의 지역별 집값 데이터셋

 - CRIM	: 지역별 범죄 발생률
 - ZN	: 25,000 평방피트를 초과하는 거주지역의 비율
 - INDUS: 비상업지역 토지의 비율
 - CHAS	: 찰스강에 대한 더미변수(강의 경계에 위치한 경우는 1, 아니면 0)
 - NOX	: 일산화질소 농도
 - RM	: 주택 1가구당 평균 방의 개수
 - AGE	: 1940년 이전에 건축된 소유주택의 비율
 - DIS	: 5개의 보스턴 고용센터까지의 접근성 지수
 - RAD	: 고속도로까지의 접근성 지수
 - TAX	: 10,000 달러 당 재산세율
 - PTRATIO : 지역별 교사 한명당 학생 비율
 - B	: 지역의 흑인 거주 비율
 - LSTAT: 하위계층의 비율(%)
 
 - MEDV	: Target.  지역의 주택가격 중앙값 (단위: $1,000)


In [ ]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

In [ ]:
# dataframe으로 만들기
cols = ["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT"]
df = pd.DataFrame(X, columns=cols)
df['MEDV'] = y
df.head()

In [ ]:
df.info()

## LinearRegression
- 가장 기본적인 선형 회귀 모델
- 각 Feauture에 가중합으로 Y값을 추론한다.
### 데이터 전처리

- **선형회귀 모델사용시 전처리**
    - **범주형 Feature**
        - : 원핫 인코딩
    - **연속형 Feature**
        - Feature Scaling을 통해서 각 컬럼들의 값의 단위를 맞춰준다.
        - StandardScaler를 사용할 때 성능이 더 잘나오는 경향이 있다.

##### X, y 분리, train/test set 나누기

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# 회귀: stratify를 설정하지 않는다.
X_train.shape, X_test.shape

##### Feature scaling

In [ ]:
# CHAS 를 제외하고 feature scaling  처리.

# X에서 chas feature를 추출
chas_train = X_train[:, 3].reshape(-1, 1)
chas_test = X_test[:, 3].reshape(-1, 1)
print(chas_train.shape, chas_test.shape)

In [ ]:
np.unique(chas_train), np.unique(chas_test)

In [ ]:
# X에서 chas feature제거
X_train = np.delete(X_train, 3, axis=1)  # (삭제할 대상 ndarray, 삭제할 index, axis=축)
X_test = np.delete(X_test, 3, axis=1)

X_train.shape, X_test.shape

In [ ]:
# scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# chas 를 추가
X_train_scaled = np.append(X_train_scaled, chas_train, axis=1)
X_test_scaled = np.append(X_test_scaled, chas_test, axis=1)

In [ ]:
print(X_train_scaled.shape, X_test_scaled.shape)

In [ ]:
X_train_scaled[:2]

In [ ]:
### 컬럼명도 재정렬하기 (CHAS를 맨 뒤로)
del cols[3]
cols.append('CHAS')

In [ ]:
cols

##### 모델 생성, 학습

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

In [ ]:
# intercept (bias-편향) 조회
lr.intercept_

In [ ]:
# coef (weight - 각 feature  곱하는 가중치) => feature가 13개 이므로 coef_도 13개로 구성됨.
print(lr.coef_.shape)
lr.coef_

> ### Coeficient의 부호
> - weight가 
> - 양수: Feature가 1 증가할때 y(집값)도 weight만큼 증가한다.
> - 음수: Feature가 1 증가할때 y(집값)도 weight만큼 감소한다.
> - 절대값 기준으로 0에 가까울 수록 집값에 영향을 주지 않고 크면 클수록(0에서 멀어질 수록) 집값에 영향을 많이 주는 Feature 란 의미가 된다.

##### 평가

In [ ]:
from metrics import print_metrics_regression as pmr

pred_train = lr.predict(X_train_scaled)
pred_test = lr.predict(X_test_scaled)

In [ ]:
print(y_train[:5])
print(pred_train[:5])

In [ ]:
pmr(y_train, pred_train, "trainset 평가")

In [ ]:
pmr(y_test, pred_test, "testset 평가")

In [ ]:
# y_test, pred_test 선그래프를 이용해 차이를 확인
import matplotlib.pyplot as plt
plt.figure(figsize=(20,7))
plt.plot(range(len(y_test)), y_test, marker='x', label='정답')  #X: index, y: 정답/추론값
plt.plot(range(len(y_test)), pred_test, marker='o', label='추론값')

plt.legend()
plt.show()


# 다항회귀 (Polynomial Regression)
- 단순한 직선형 보다 복잡한 비선형의 데이터셋을 학습하기 위한 방식.
    - Feature가 너무 적어 y의 값들을 다 표현 하지 못하여 underfitting이 된 경우 Feature를 늘려준다.
- 각 Feature들을 거듭제곱한 것과 Feature들 끼리 곱한 새로운 특성들을 추가한 뒤 선형모델로 훈련시킨다.
    - 파라미터 가중치를 기준으로는 일차식이 되어 선형모델이다. 파라미터(Coef, weight)들을 기준으로는 N차식이 되어 비선형 데이터를 추론할 수 있는 모델이 된다.
- `PolynomialFeatures` Transformer를 사용해서 변환한다.

## 예제

##### 데이터셋 만들기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)

def func(X):
    return X**2 + X + 2 + np.random.normal(0,1, size=(X.size, 1))

m = 100 # 데이터개수
X = 6 * np.random.rand(m, 1) - 3
y = func(X)
y = y.flatten()

print(X.shape, y.shape)

In [ ]:
df = pd.DataFrame({"X":X.flatten(), "y":y})
df.head()

In [ ]:
plt.figure(figsize=(7,6))
plt.scatter(X, y)
plt.show()

##### 모델생성, 학습

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X, y)

In [ ]:
pmr(y, lr.predict(X))

In [ ]:
X_new = np.linspace(-3, 3, 100).reshape(-1, 1)
pred_new = lr.predict(X_new)

plt.scatter(X, y)
plt.plot(X_new, pred_new, color='red')
plt.show()

##### PolynomialFeatures를 이용해 다항회귀구현

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# 기존 컬럼을 N 제곱한 컬럼과 기존 컬럼끼리 곱한 컬럼을 추가로 생성하는 transformer. 
pn = PolynomialFeatures(degree=2,  # 최고차항의 차수를 지정->N제곱 N을 지정.  2-> X, X**2  / 3-> X, X**2, X**3
                        include_bias=False)  # 상수항(모든값이 1인 feature)를 생성하지 않는다. (default: True)
X_poly = pn.fit_transform(X)
print(X.shape, X_poly.shape)

In [ ]:
X[:5]

In [ ]:
X_poly[:5]

In [ ]:
# 변환된 dataset의 feature들이 어떻게 만들어 졌는지를 출력
pn.get_feature_names_out()

##### LinearRegression 모델을 이용해 평가

In [ ]:
lr2 = LinearRegression()
lr2.fit(X_poly, y)

print(lr2.coef_, lr2.intercept_)

In [ ]:
pred2 = lr2.predict(X_poly)
pmr(y, pred2)

##### 시각화

In [ ]:
X_new = np.linspace(-3, 3, 100).reshape(-1, 1)
print(X_new.shape)
X_new_poly = pn.transform(X_new)
pred_new = lr2.predict(X_new_poly)

In [ ]:

plt.figure(figsize=(7,6))
plt.scatter(X, y)
plt.plot(X_new, pred_new, color='red')
plt.show()

## degree를 크게
- Feature가 너무 많으면 Overfitting 문제가 생긴다.

In [ ]:
pn2 = PolynomialFeatures(degree=35, include_bias=False)
X_poly2 = pn2.fit_transform(X)
print(X.shape, X_poly2.shape)  # X, X**2, X**3, X**4, X**5 ...... X**35
pn2.get_feature_names_out()

In [ ]:
lr3 = LinearRegression()
lr3.fit(X_poly2, y)
print(lr3.coef_, lr3.intercept_)

In [ ]:
# 검증
pmr(y, lr3.predict(X_poly2))  # 결과가 별차이 안나는 이유는 train set으로 검증했기 때문.

In [ ]:
# 시각화
X_new_poly2 = pn2.transform(X_new)  # X_new: -3 ~ 3 100등분
print(X_new_poly2.shape)
pred_new2 = lr3.predict(X_new_poly2)

plt.figure(figsize=(7,6))
plt.scatter(X, y)
plt.plot(X_new, pred_new2, color='red')

plt.ylim(-15, 20)
plt.show()

In [ ]:
# x가 3일때 추론결과값
a = pn2.transform(np.array([[3]]))
lr3.predict(a)

### PolynomialFeatures 예제

In [ ]:
# 여러 Feature를 가진(다차원) Dataset 변환
import numpy as np

data = np.arange(12).reshape(4,3)
print(data.shape)
data

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pn = PolynomialFeatures(degree=2,  # 최고차항 설정
#                         include_bias=False # 상수항 추가 여부. True: 추가(기본), False: 추가하지 않는다.
                       )
data_poly = pn.fit_transform(data)
data_poly.shape

In [ ]:
# 변환된 Feature가 어떻게 구성되었는지 확인
# feature가 여러개인 경우: 각 feature들을 제곱한 것과 feature끼리 교차곱한 feature들이 생성된다.
pn.get_feature_names_out()

In [ ]:
data_poly

### PolynomialFeatures를 Boston Dataset에 적용

In [ ]:
pn = PolynomialFeatures(degree=2, include_bias=False)
X_train_scaled_poly = pn.fit_transform(X_train_scaled)
X_test_scaled_poly = pn.transform(X_test_scaled)

In [ ]:
print(X_train_scaled.shape, X_train_scaled_poly.shape)

In [ ]:
print(pn.get_feature_names_out())

##### 모델 생성 학습 추론 평가

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_scaled_poly, y_train)
print(lr.intercept_)
print(lr.coef_)

In [ ]:
# 검증
pred_train = lr.predict(X_train_scaled_poly)
pred_test = lr.predict(X_test_scaled_poly)

In [ ]:
from metrics import print_metrics_regression as pmr

pmr(y_train, pred_train, "Degree=2, Train set 평가")
pmr(y_test, pred_test, "Degree=2, Test set 평가")

In [ ]:
pn2 = PolynomialFeatures(degree=5)
X_train_scaled_poly2 = pn2.fit_transform(X_train_scaled)
X_test_scaled_poly2 = pn2.transform(X_test_scaled)

print(X_train_scaled_poly2.shape)

lr2 = LinearRegression()
lr2.fit(X_train_scaled_poly2, y_train)

pred_train2 = lr2.predict(X_train_scaled_poly2)
pred_test2 = lr2.predict(X_test_scaled_poly2)

In [ ]:
pmr(y_train, pred_train2, "Degree 5, Train set평가")
pmr(y_test, pred_test2, "Degree 5, Test set 평가")

## 규제 (Regularization)
- 선형 회귀 모델에서 과대적합(Overfitting) 문제를 해결하기 위해 가중치(회귀계수)에 페널티 값을 적용한다.
- 입력데이터의 Feature들이 너무 많은 경우 Overfitting이 발생.
    - Feature수에 비해 관측치 수가 적은 경우 모델이 복잡해 지면서 Overfitting이 발생한다.
- 해결
    - 데이터를 더 수집한다. 
    - Feature selection
        - 불필요한 Features들을 제거한다.
    - 규제 (Regularization) 을 통해 Feature들에 곱해지는 가중치가 커지지 않도록 제한한다.(0에 가까운 값으로 만들어 준다.)
        - LinearRegression의 규제는 학습시 계산하는 오차를 키워서 모델이 오차를 줄이기 위해 가중치를 0에 가까운 값으로 만들도록 하는 방식을 사용한다.
        - L1 규제 (Lasso)
        - L2 규제 (Ridge)
    

## Ridge Regression (L2 규제)
- 손실함수(loss function)에 규제항으로 $\alpha \sum_{i=1}^{n}{w_{i}^{2}}$ (L2 Norm)을 더해준다.
- $\alpha$는 하이퍼파라미터로 모델을 얼마나 많이 규제할지 조절한다. 
    - $\alpha = 0$ 에 가까울수록 규제가 약해진다. (0일 경우 선형 회귀동일)
    - $\alpha$ 가 커질 수록 모든 가중치가 작아져 입력데이터의 Feature들 중 중요하지 않은 Feature의 예측에 대한 영향력이 작아지게 된다.

$$
\text{손실함수}(w) = \text{MSE}(w) + \alpha \cfrac{1}{2}\sum_{i=1}^{n}{w_{i}^{2}}
$$

> **손실함수(Loss Function):** 모델의 예측한 값과 실제값 사이의 차이를 정의하는 함수로 모델이 학습할 때 사용된다.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
for alpha in [0.1, 1, 10, 1000]:
    ridge = Ridge(alpha=alpha, random_state=0)   # 규제 하이퍼파라미터: alpha - 클수록 규제가 커진다. (더 단순한 모델). 기본값: 1
    ridge.fit(X_train_scaled, y_train)  
    pred_train1 = ridge.predict(X_train_scaled)
    pred_test1 = ridge.predict(X_test_scaled)

    print(f'alpha: {alpha}')
    pmr(y_train, pred_train1, "train set")
    pmr(y_test, pred_test1, 'test set')
    print("="*100)

## GridSearchCV를 이용해 최적의 alpha 탐색

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    "alpha":[0, 0.1, 1, 10, 20, 30, 40, 50, 100]
}
ridge = Ridge(random_state=0)
gs = GridSearchCV(ridge, params, scoring=['r2', 'neg_mean_squared_error'], refit='r2', cv=4)

gs.fit(X_train_scaled, y_train)

In [ ]:
print("best score:", gs.best_score_)
print('best alpha:', gs.best_params_)

In [ ]:
import pandas as pd
result = pd.DataFrame(gs.cv_results_)
result.head()

## 규제 alpha 에 따른 weight 변화

In [ ]:
import matplotlib.pyplot as plt

alpha_list = [0, 1, 10, 100, 500, 1000, 3000]
coef_df = pd.DataFrame()  # alpha별 각 Feature의 weight를 저장할 DataFrame

plt.figure(figsize=(6,30))

for idx, alpha in enumerate(alpha_list, start=1):
    ridge = Ridge(alpha=alpha, random_state=0)
    ridge.fit(X_train_scaled, y_train)
    
    weights = pd.Series(np.round(ridge.coef_, 3))
    coef_df[f'alpha {alpha}'] = weights
    
    w = weights.sort_values()
    plt.subplot(7, 1, idx)
    plt.bar(x=w.index, height=w)
    plt.title(f'alpha-{alpha}')
    plt.yticks(range(-4,3))
    plt.grid(True)
    
plt.tight_layout()
plt.show()

In [ ]:
coef_df

## Lasso(Least Absolut Shrinkage and Selection Operator) Regression (L1 규제)

- 손실함수에 규제항으로 $\alpha \sum_{i=1}^{n}{\left| w_i \right|}$ (L1 Norm)더한다.
- Lasso 회귀의 상대적으로 덜 중요한 특성의 가중치를 0으로 만들어 자동으로 Feature Selection이 된다.

$$
\text{손실함수}(w) = \text{MSE}(w) + \alpha \sum_{i=1}^{n}{\left| w_i \right|}
$$

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso(random_state=0)  #alpha : 기본값 - 1
lasso.fit(X_train_scaled, y_train)

pred_train = lasso.predict(X_train_scaled)
pred_test = lasso.predict(X_test_scaled)

print('alpha: 1')
pmr(y_train, pred_train, "train")
pmr(y_test, pred_test, "test")

In [ ]:
lasso.coef_

In [ ]:
lasso = Lasso(alpha=10, random_state=0)  #alpha : 10
lasso.fit(X_train_scaled, y_train)

pred_train = lasso.predict(X_train_scaled)
pred_test = lasso.predict(X_test_scaled)

print('alpha: 10')
pmr(y_train, pred_train, "train")
pmr(y_test, pred_test, "test")

In [ ]:
lasso.coef_

In [ ]:
lasso.intercept_

In [ ]:
pred_test

In [ ]:
# alpha 변화에 따른 weight 변화
alpha_list = [0, 0.01, 0.1, 0.5, 1, 10]

lasso_coef_df = pd.DataFrame()
plt.figure(figsize=(7,30))

for idx, alpha in enumerate(alpha_list, start=1):
    lasso = Lasso(alpha=alpha, random_state=0)
    lasso.fit(X_train_scaled, y_train)
    
    lasso_coef_df[f'alpha {alpha}'] = lasso.coef_
    
    plt.subplot(6, 1, idx)
    plt.bar(x=range(len(lasso.coef_)), height=lasso.coef_)
    plt.title(f"Lasso alpha-{alpha}")
    plt.grid(True)
    
plt.tight_layout()
plt.show()

In [ ]:
lasso_coef_df

### PolynormialFeatures로 전처리한 Boston Dataset 에 Ridge, Lasso 규제 적용

In [ ]:
# degree=2
X_train_scaled_poly.shape

In [ ]:
alpha_list = [0.01, 0.1, 1, 10, 50, 100]  #Ridge, Lasso에서 사용할 alpha list

##### LinearRegression으로 평가

In [ ]:
# 모델 생성
lr = LinearRegression()
# 학습
lr.fit(X_train_scaled_poly, y_train)
# 검증
pmr(y_train, lr.predict(X_train_scaled_poly), "LinearRegression train set")
pmr(y_test, lr.predict(X_test_scaled_poly), "LinearRegression test set")

##### Ridge 의 alpha값 변화에 따른 R square 확인

In [ ]:
from sklearn.metrics import r2_score

ridge_train_r2 = []
ridge_test_r2 = []

for alpha in alpha_list:
    #모델생성
    ridge = Ridge(alpha=alpha, random_state=0)
    #학습
    ridge.fit(X_train_scaled_poly, y_train)
    # 검증
    ridge_train_r2.append(r2_score(y_train, ridge.predict(X_train_scaled_poly)))
    ridge_test_r2.append(r2_score(y_test, ridge.predict(X_test_scaled_poly)))
                          

In [ ]:
ridge_df = pd.DataFrame({
    "alpha":alpha_list,
    "train r2":ridge_train_r2,
    "test r2":ridge_test_r2
})
ridge_df

##### lasso 의 alpha값 변화에 따른 R square 확인

In [ ]:
from sklearn.metrics import r2_score

lasso_train_r2 = []
lasso_test_r2 = []

for alpha in alpha_list:
    #모델생성
    lasso = Lasso(alpha=alpha, random_state=0)
    #학습
    lasso.fit(X_train_scaled_poly, y_train)
    # 검증
    lasso_train_r2.append(r2_score(y_train, lasso.predict(X_train_scaled_poly)))
    lasso_test_r2.append(r2_score(y_test, lasso.predict(X_test_scaled_poly)))
                          

In [ ]:
lasso_df = pd.DataFrame({
    "alpha":alpha_list,
    "train r2":lasso_train_r2,
    "test r2":lasso_test_r2
})
lasso_df

## ElasticNet(엘라스틱넷)
- 릿지와 라쏘를 절충한 모델.
- 규제항에 릿지, 라쏘 규제항을 더해서 추가한다. 
- 혼합비율 $r$을 사용해 혼합정도를 조절
- $r=0$이면 릿지와 같고 $r=1$이면 라쏘와 같다.

$$
\text{손실함수}(w) = \text{MSE}(w) + r\alpha \sum_{i=1}^{n}{\left| w_i \right|}  + \cfrac{1-r}{2}\alpha\sum_{i=1}^{n}{w_{i}^{2}}
$$

In [ ]:
from sklearn.linear_model import ElasticNet
e_net = ElasticNet(alpha=0.1, l1_ratio=0.6, random_state=0)
e_net.fit(X_train_scaled_poly, y_train)

pmr(y_train, e_net.predict(X_train_scaled_poly), "train set")
pmr(y_test, e_net.predict(X_test_scaled_poly), "test set")

# 정리
- 일반적으로 선형회귀의 경우 어느정도 규제가 있는 경우가 성능이 좋다.
- 기본적으로 **Ridge**를 사용한다.
- Target에 영향을 주는 Feature가 몇 개뿐일 경우 특성의 가중치를 0으로 만들어 주는 **Lasso** 사용한다. 
- 특성 수가 학습 샘플 수 보다 많거나 feature간에 연관성이 높을 때는 **ElasticNet**을 사용한다.